In [31]:
import pandas as pd
import ast
import numpy as np
import json
from pandas.io.json import json_normalize
import collections as c

In [4]:
df = pd.read_csv('raw/train.csv')
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [5]:
## Extracts Json objects to lists prioitizing names
features_to_fix=['belongs_to_collection', 'genres', 'production_companies','production_countries','spoken_languages','Keywords']
for feature in features_to_fix:
    df.loc[df[feature].notnull(),feature]=df.loc[df[feature].notnull(),feature].apply(lambda x : ast.literal_eval(x)).apply(lambda x : [y["name"] for y in x])

In [6]:
## Extracts jason objects to dictionaries for cast and crew with 
df.loc[df["cast"].notnull(),"cast"]=df.loc[df["cast"].notnull(),"cast"].apply(lambda x : ast.literal_eval(x))
df.loc[df["crew"].notnull(),"crew"]=df.loc[df["crew"].notnull(),"crew"].apply(lambda x : ast.literal_eval(x))

In [7]:
## indicated whether a movie has a website
BinaryMap = {True:0, False:1}
df['hasWebpage']= df.homepage.isna().map(BinaryMap)
df.head()


,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue,hasWebpage
0,1,[Hot Tub Time Machine Collection],14000000,[Comedy],NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,93.0,[English],Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[time travel, sequel, hot tub, duringcreditsst...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651,0
1,2,[The Princess Diaries Collection],40000000,"[Comedy, Drama, Family, Romance]",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,113.0,[English],Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[coronation, duty, marriage, falling in love]","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435,0
2,3,NaN,3300000,[Drama],http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,105.0,[English],Released,The road to greatness can take you to the edge.,Whiplash,"[jazz, obsession, conservatory, music teacher,...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000,1
3,4,NaN,1200000,"[Thriller, Drama]",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,122.0,"[English, हिन्दी]",Released,NaN,Kahaani,"[mystery, bollywood, police corruption, crime,...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000,1
4,5,NaN,0,"[Action, Thriller]",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,118.0,[한국어/조선말],Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970,0


In [8]:
##handles missing values for spoken language  based on original language
df.loc[(df['spoken_languages'].isnull()) & (df['original_language']=='ru'),'spoken_languages']= ['Pусский']
df.loc[(df['spoken_languages'].isnull()) & (df['original_language']=='fr'),'spoken_languages']= ['Français']
df.loc[(df['spoken_languages'].isnull()) & (df['original_language']=='en'),'spoken_languages']= ['English']

In [9]:
def catInput(inputDF, outputDF, cols):
    for col in cols:
        for row in range(len(inputDF)):
            if type(inputDF.loc[row,col])=='list':
                for entry in inputDF.loc[row,col]:
                    outputDF.loc[row,entry]=1
            else:
                outputDF.loc[row,inputDF.loc[row,col]]=1

def catExtract(inputDF, cols):
    output=list()
    for col in cols:
        for entry in inputDF[col]:
                if(type(entry)==list):
                    output.extend(entry)
                else:
                    output.append(entry)
        
    output= set(output)
    return output

catUsed = catExtract(df, features_to_fix)
cleanedCat=pd.DataFrame(np.zeros([len(df),len(catUsed)]),columns=catUsed)
print(cleanedCat.shape)
catInput(inputDF = df, outputDF = cleanedCat, cols=features_to_fix)


(3000, 11668)


In [87]:
temp = json_normalize(df.loc[4,'cast'])
print(temp)

   cast_id                character                 credit_id  gender  \
0        3                 Chun-soo  52fe464b9251416c75073b47       2   
1        4   Company president Kang  52fe464b9251416c75073b4b       2   
2        5                    Yu-ri  52fe464b9251416c75073b4f       2   
3        7  Company president Hwang  52fe464b9251416c75073b59       2   

        id          name  order                      profile_path  
0    84751  Kim Kang-woo      0  /lJvcgUx1qEnNfGcCtbmp3EC5vCG.jpg  
1    64453  Jo Jae-hyeon      1  /2QVUP8t3zmKcwlm7IBOzPVikKe7.jpg  
2    84752  Park Si-yeon      2  /pIWhmT2GTVcaNTb6Qta2KS9jyTT.jpg  
3  1130534  Kim Joon-bae      3  /pMy3HSxj6DdysI1P2tpjcD4FyWM.jpg  


In [83]:
rowsCrewCleaned = []

for i in range(len(df)):
    genderDict = c.defaultdict(int)
    departmentDict = c.defaultdict(int)
    jobDict = c.defaultdict(list)
    try:
        currentNormalizedata = json_normalize(df.loc[i,'crew'])
        genderList = currentNormalizedata['gender'].tolist()
        for entry in genderList:
            genderDict[entry]+=1

        gender =['crew_unknown','crew_female', 'crew_male']
        for counter, value in enumerate(gender):
            if counter in genderDict:
                genderDict[value] = genderDict.pop(counter)
        departmentList = currentNormalizedata['department'].tolist()
        for entry in departmentList:
            departmentDict[entry]+=1
    
        jobList = zip(currentNormalizedata['job'].tolist(), currentNormalizedata['name'].tolist())
        for k, v in jobList:
            jobDict[k].append(v)
    except:
        pass
    
        
    crewCombine= {**genderDict,**departmentDict,**jobDict}        
    rowsCrewCleaned.append(crewCombine)
len(rowsCrewCleaned)

3000

In [85]:
cleanedCrew = pd.DataFrame.from_records(rowsCrewCleaned)
cleanedCrew.head()

,24 Frame Playback,2D Artist,2D Supervisor,3D Animator,3D Artist,3D Coordinator,3D Generalist,3D Modeller,3D Supervisor,ADR & Dubbing,...,Visual Effects Technical Director,Vocal Coach,Wardrobe Supervisor,Wig Designer,Wigmaker,Writer,Writing,cast_female,cast_male,cast_unknown
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[Josh Heald],2.0,NaN,13.0,59.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,4.0,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,11.0,49.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[Sujoy Ghosh],1.0,NaN,NaN,3.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[Jong-seok Yoon],1.0,NaN,NaN,2.0


In [98]:
currentNormalizedata = json_normalize(df.loc[2,'cast'])
print(currentNormalizedata['name'].tolist())

['Miles Teller', 'J.K. Simmons', 'Melissa Benoist', 'Austin Stowell', 'Jayson Blair', 'Kavita Patil', 'Paul Reiser', 'Nate Lang', 'Chris Mulkey', 'Damon Gupton', 'Suanne Spoke', 'Max Kasch', 'Charlie Ian', 'Kofi Siriboe', 'C.J. Vana', 'Tarik Lowe', 'Tyler Kimball', 'Rogelio Douglas Jr.', 'Adrian Burks', 'Joseph Bruno', 'Michael D. Cohen', 'Jocelyn Ayanna', 'Keenan Henson', 'Janet Hoskins', 'April Grace', "Clifton 'Fou Fou' Eddie", 'Calvin C. Winbush', 'Marcus Henderson', 'Tony Baker', 'Henry G. Sanders', 'Sam Campisi', 'Jimmie Kirkpatrick', 'Keenan Allen', 'Ayinde Vaughan', 'Shai Golan', 'Yancey Wells', 'Candace Roberge', 'Krista Kilber', 'Cici Leah Campbell', 'Damien Coates', 'Kyle Julian Graham', 'Ellee Jane Hounsell', 'Stephen Hsu', 'Herman Johansen', 'Wendee Lee', 'Dakota Lupo', 'Jesse Mitchell', 'Amanda Newman', 'Joseph Oliveira', 'Michelle Ruff', 'Daniel Weidlein']


In [99]:
rowsCastClean =[]
for i in range(len(df)):
    genderDict = c.defaultdict(int)
    actors = dict()
    try:
        currentNormalizedata = json_normalize(df.loc[i,'cast'])
        genderList = currentNormalizedata['gender'].tolist()
        for entry in genderList:
            genderDict[entry]+=1

        gender =['crew_unknown','crew_female', 'crew_male']
        for counter, value in enumerate(gender):
            if counter in genderDict:
                genderDict[value] = genderDict.pop(counter)
        actors['actors_names']=currentNormalizedata['name'].tolist()
    except:
        pass
    castCombine ={**genderDict, **actors}
    rowsCastClean.append(castCombine)


In [100]:
castCleaned = pd.DataFrame.from_records(rowsCastClean)

In [101]:
castCleaned.head()

,actors_names,crew_female,crew_male,crew_unknown
0,"[Rob Corddry, Craig Robinson, Clark Duke, Adam...",8.0,10.0,6.0
1,"[Anne Hathaway, Julie Andrews, H√©ctor Elizond...",10.0,10.0,NaN
2,"[Miles Teller, J.K. Simmons, Melissa Benoist, ...",7.0,13.0,31.0
3,"[Vidya Balan, Nawazuddin Siddiqui, Parambrata ...",1.0,2.0,4.0
4,"[Kim Kang-woo, Jo Jae-hyeon, Park Si-yeon, Kim...",NaN,4.0,NaN
